## The Poisson as a limiting case of the Binomial

Consider a Binomial process with $N$ trials and success probability $p$. We wish to consider the situation where $N \to \infty$ and $p \to 0$ but $N p = \lambda$, a constant. In this case, the probability of $k$ successes in $N$ trials is:
\begin{align*}
{N \choose k} p^k (1 - p)^{N - k} & = \frac{N!}{k! (N - k)!} p^k (1 - p)^{N - k} \\
    & = \frac{1}{k!} \cdot \frac{(N - k + 1) (N - k + 2) \cdots N}{N^k} 
        \cdot (Np)^k 
        \cdot \left ( 1 - \frac{\lambda}{N} \right )^{N - k} \\
    & = \frac{\lambda^k}{k!} \left ( 1 - \frac{k - 1}{N} \right ) 
        \left ( 1 - \frac{k - 2}{N} \right ) \frac{N}{N} 
        \cdot \frac{1}{(1 - \lambda / N)^k} \cdot \left ( 1 - \frac{\lambda}{N}\right )^N 
\end{align*}
As $N \to \infty$ and $p \to 0$, the above approaches $\lambda^k e^{-\lambda} / k!$, which is the pmf of the Poisson distribution.